<a href="https://colab.research.google.com/github/DrAlexSanz/nlp-demystified/blob/main/Fake_news_detection_RNN_KERAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import nltk
import numpy as np
import requests
import tensorflow as tf
import pandas as pd
import spacy
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
#from nltk.corpus import treebank, brown, conll2000
from sklearn.model_selection import train_test_split
from tensorflow import keras
from google.colab import drive

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
drive.mount('/content/drive/')

Mounted at /content/drive/


## Import data
Easier to download from kaggle and upload to my drive in this case

In [44]:
raw_train = pd.read_csv("/content/drive/MyDrive/Datasets/fake_news/train.csv", nrows = 1000) # Otherwise it will never finish tokenizing or training

raw_train["text"] = raw_train["text"].astype(str) # so the tokenizer doesn't complain when it finds a number. Alternatively, remove all the numbers.
#raw_test = pd.read_csv("/content/drive/MyDrive/Datasets/fake_news/test.csv") # Test dataset is to send to kaggle originally. I don't need this here.
raw_train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [45]:
text = raw_train["text"]
labels = raw_train["label"]

x_train, y_train, x_test, y_test = train_test_split(text, labels, test_size = 0.15)

In [46]:
nlp = spacy.load("en_core_web_sm")

def spacy_tokenizer(doc):
  # use .lemma_ instead of .text to reduce memory footprint
  transformed = [t.text for t in nlp(doc) if \
                not t.is_space and \
                not t.is_stop and \
                t.is_alpha]

  return transformed

In [47]:
sentence_tokenizer = keras.preprocessing.text.Tokenizer(oov_token='', lower = True, analyzer = spacy_tokenizer) # The keras tokenizer is very simple and it works for many use cases. I can improve it a bit.
sentence_tokenizer.fit_on_texts(x_train) # Since the fitting is using the spacy tokenizer it will take much longer.

KeyboardInterrupt: ignored

In [ ]:
# And see how big is the vocabulary. Probably it would be a good idea to lemmatize or put a word limit because 238K words is not too smart
# Also, the keras tokenizer leaves all the 's and weird characters as unicode. I can probably solve it better but at least I will use the spacy tokenizer.

print(f"The vocabulary size is: {len(sentence_tokenizer.word_index)}")

In [ ]:
sentence_tokenizer.get_config()